# Qubit Tapering 
### in the Stabilizer Subspace Projection formalism
Here, we take a look at the qubit reduction technique of [tapering](https://arxiv.org/abs/1701.08213) and an implementation based on the core `S3_projection` class. Unlike [Contextual-Subspace VQE](https://doi.org/10.22331/q-2021-05-14-456), this technique is *exact*, in the sense that it perfectly preserves the energy spectrum of the input operator.

At the core of qubit tapering is a symmetry of the Hamiltonian, which in this case means a set of universally commuting operators. The idea is that these operators must be simultaneously measureable and so can be treated independently of the remaining Hamiltonian terms. The method works by finding an independent generating set for the symmetry and seeks to find the 'correct' assignment of eigenvalues (called a *sector*), which completely determines the measurement outcome of the symmetry operators. Once this is obtained, the theory of stabilizers allows us to rotate the symmetry generators onto single Pauli $X$ operators, and since they must commute universally every operator of the rotated Hamiltonian will consist of an identity or Pauli $X$ in the corresponding qubit position. This means we can drop the qubit from the Hamiltonian, leaving in its place the eigenvalue determined by the chosen sector.

In [1]:
from symmer.symplectic import PauliwordOp, QuantumState
from symmer.projection import QubitTapering
from symmer.chemistry  import MoleculeBuilder, exact_gs_energy
import numpy as np
from matplotlib import pyplot as plt

First, we shall construct a molecule using OpenFermion with PySCF the underlying quantum chemistry package. The resulting fermionic operator will be mapped onto qubits via the Jordan-Wigner transformation.

In [10]:
# Two examples are provided -- comment out as necessary

# One for which Hartree-Fock identifies the correct symmetry sector...
#geometry=[
#    ("O",(0,0,0)),
#    ("H",(0.952519,0,0)),
#    ("H",(-0.246530058,0.9200627021,0))
#]

# ...and one for which it does not.
geometry=[
    ('H', (0.0,0.0,0.0)),
    ('H', (2.454,0.0,0.0)),
    ('H', (2.454,2.454,0.0)),
    ('H', (0.0,2.454,0.0))
     ]    
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

Molecule geometry:
H	0.0	0.0	0.0
H	2.454	0.0	0.0
H	2.454	2.454	0.0
H	0.0	2.454	0.0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -1.333349777097311
MP2 energy:  -1.6161399127016054
CCSD energy: -1.8261847346725237
FCI energy:  -1.8743019741837195


Number of qubits: 8


In [11]:
print('Jordan-Wigner Hamiltonian:\n')
print(molecule.H_q)

Jordan-Wigner Hamiltonian:

-1.081+0.000j IIIIIIII +
 0.065+0.000j ZIIIIIII +
-0.014+0.000j YZZZZZYI +
-0.014+0.000j XZZZZZXI +
 0.065+0.000j IZIIIIII +
-0.014+0.000j IYZZZZZY +
-0.014+0.000j IXZZZZZX +
 0.038+0.000j IIZIIIII +
 0.038+0.000j IIIZIIII +
 0.038+0.000j IIIIZIII +
 0.038+0.000j IIIIIZII +
 0.014+0.000j IIIIIIZI +
 0.014+0.000j IIIIIIIZ +
 0.091+0.000j ZZIIIIII +
 0.014+0.000j YIZZZZYI +
 0.014+0.000j XIZZZZXI +
 0.019+0.000j YXXYIIII +
-0.019+0.000j YYXXIIII +
-0.019+0.000j XXYYIIII +
 0.019+0.000j XYYXIIII +
 0.057+0.000j YXIIXYII +
-0.057+0.000j YYIIXXII +
-0.057+0.000j XXIIYYII +
 0.057+0.000j XYIIYXII +
 0.014+0.000j ZYZZZZZY +
 0.014+0.000j ZXZZZZZX +
 0.024+0.000j YXIIIIXY +
-0.024+0.000j YYIIIIXX +
-0.024+0.000j XXIIIIYY +
 0.024+0.000j XYIIIIYX +
 0.050+0.000j ZIZIIIII +
 0.007+0.000j YZIZZZYI +
 0.007+0.000j XZIZZZXI +
 0.069+0.000j ZIIZIIII +
-0.027+0.000j YZZIZZYI +
-0.027+0.000j XZZIZZXI +
 0.033+0.000j YZYYZZZY +
 0.033+0.000j YZYXZZZX +
 0.033+0.000j XZXYZZZY

In [12]:
molecule.H_q.to_dictionary

{'IIIIIIII': (-1.0811399195666886+0j),
 'IIIIIIIZ': (0.013810914373994614+0j),
 'IIIIIIZI': (0.013810914373994537+0j),
 'IIIIIIZZ': (0.09424143717952177+0j),
 'IIIIIZII': (0.03758180217247291+0j),
 'IIIIIZIZ': (0.04955248667420177+0j),
 'IIIIIZZI': (0.07052407412802449+0j),
 'IIIIZIII': (0.037581802172472846+0j),
 'IIIIZIIZ': (0.07052407412802449+0j),
 'IIIIZIZI': (0.04955248667420177+0j),
 'IIIIZZII': (0.11637394770154717+0j),
 'IIIZIIII': (0.037581802172472895+0j),
 'IIIZIIIZ': (0.04186848978406614+0j),
 'IIIZIIZI': (0.10114884701928667+0j),
 'IIIZIZII': (0.05356550494495824+0j),
 'IIIZZIII': (0.053636244956641634+0j),
 'IIZIIIII': (0.03758180217247281+0j),
 'IIZIIIIZ': (0.10114884701928667+0j),
 'IIZIIIZI': (0.04186848978406614+0j),
 'IIZIIZII': (0.053636244956641634+0j),
 'IIZIZIII': (0.05356550494495824+0j),
 'IIZZIIII': (0.11637394770154719+0j),
 'IZIIIIII': (0.06458158745266908+0j),
 'IZIIIIIZ': (0.053811564684024295+0j),
 'IZIIIIZI': (0.07737596674316873+0j),
 'IZIIIZII': (0.04

We are now in a position to initialize our `QubitTapering` class, which will identify a set of independent operators $\mathcal{S}$ that generate the Hamiltonian symmetry. Since the set is independent, there will exist a Clifford rotation $U$ mapping the elements $S \in \mathcal{S}$ to single-qubit Pauli operators, i.e. $USU^* = \sigma_p^{(i)}$ for some qubit position $i$ and $p \in \{1,2,3\}$ corresponding with Pauli $\{X,Y,Z\}$ operators; this observation that each Hamiltonian symmetry contributes a qubits-worth degree of freedom is at the core of Qubit Tapering. Since $S$ commutes with each term of $H$ by definition, the single-qubit Pauli $\sigma_p^{(i)}$ must commute with the rotated Hamiltonian $UHU^*$, meaning each term therein must consist either of identity or $\sigma_p$ in the qubit position indexed by $i$. As such, the qubit may be dropped from the Hamiltonian, leaving in its place a $\pm1$ eigenvalue. In our implementation, the above Clifford rotation is constructed as a sequence of $\frac{\pi}{2}$ rotations $U_k = e^{i \frac{\pi}{4} R_k}$, yielding $U = \prod_{k=1}^{|\mathcal{S}|} U_k$. See below for this example:

In [13]:
from symmer.symplectic import StabilizerOp

StabilizerOp.symmetry_basis(molecule.H_q, commuting_override=True)

 1 IIZZIIII 
 1 IIIIZZII 
 1 ZIIZIZZI 
 1 IZIZIZIZ

In [14]:
taper_hamiltonian = QubitTapering(molecule.H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated onto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
print('\nvia a sequence of rotations e^{i pi/4 R} where\n')
for index, (rot, angle) in enumerate(taper_hamiltonian.stabilizers.stabilizer_rotations):
    print(f'R_{index} = {rot}')

We are able to taper 4 qubits from the Hamiltonian.

The symmetry generators are

 1 IIIIZZII 
 1 IIZZIIII 
 1 IZIZIZIZ 
 1 ZIIZIZZI

and may be rotated onto the single-qubit Pauli operators

-1 IIIIXIII 
-1 IIXIIIII 
-1 IXIIIIII 
-1 XIIIIIII

via a sequence of rotations e^{i pi/4 R} where

R_0 =  1.000+0.000j IIIIYZII
R_1 =  1.000+0.000j IIYZIIII
R_2 =  1.000+0.000j IYIZIZIZ
R_3 =  1.000+0.000j YIIZIZZI


In order to perform the stabilizer subspace projection, we must also supply a symmetry sector or reference state. Under the Jordan-Wigner transformation, the Hartree-Fock state for our $M$-electron, $N$-orbital molecular system with charge=0 and multiplicity=1 will be 

$$|\mathrm{HF}\rangle = |\underbrace{1 \dots 1}_{M \,\text{times}}\; \underbrace{0 \dots 0}_{N-M \,\text{times}} \rangle.$$

Note that OpenFermion fills orbital occupations from the left... this will not always be the case! For example, if using Qiskit or some other quantum computing package the Hartree-Fock state will not look the same.

In [15]:
hf_string = ''.join([str(i) for i in molecule.hf_array])
print(f'The Hartree-Fock state is |{hf_string}>')

The Hartree-Fock state is |11110000>


The corresponding sector is obtained by measuring each symmetry generator with respect to the reference state, yielding a $\pm1$ eigenvalue assignment.

In [16]:
taper_hamiltonian.stabilizers.update_sector(molecule.hf_array)
print(f'The symmetry sector corresponding with the reference state is {taper_hamiltonian.stabilizers.coeff_vec}')

The symmetry sector corresponding with the reference state is [1 1 1 1]


This is everything we need to go ahead and perform the tapering process, which is effected by the `taper_it()` method that calls on the parent `S3_projection` class.

In [17]:
ham_tap = taper_hamiltonian.taper_it(ref_state=molecule.hf_array)
print('Tapered Hamiltonian:\n')
print(ham_tap)

Tapered Hamiltonian:

-0.848+0.000j IIII +
 0.014+0.000j IIIZ +
 0.014+0.000j IIZI +
 0.185+0.000j IIZZ +
 0.075+0.000j IZII +
 0.239+0.000j IZIZ +
 0.239+0.000j IZZI +
 0.075+0.000j ZIII +
 0.285+0.000j ZIIZ +
 0.285+0.000j ZIZI +
 0.322+0.000j ZZII +
 0.065-0.000j ZZIZ +
 0.065-0.000j ZZZI +
 0.155+0.000j ZZZZ +
 0.026+0.000j IIIX +
 0.014-0.000j IIZX +
-0.041+0.000j IZZX +
 0.041+0.000j ZIZX +
-0.026+0.000j ZZIX +
-0.014+0.000j ZZZX +
-0.026+0.000j IIXI +
-0.014+0.000j IIXZ +
 0.041+0.000j IZXZ +
-0.041+0.000j ZIXZ +
 0.026+0.000j ZZXI +
 0.014+0.000j ZZXZ +
-0.047+0.000j IIYY +
 0.047+0.000j ZZYY +
-0.057+0.000j IXII +
-0.057+0.000j IXZZ +
 0.057-0.000j ZXIZ +
 0.057+0.000j ZXZI +
-0.035+0.000j IXIX +
-0.035+0.000j IYIY +
 0.035+0.000j ZXZX +
 0.035+0.000j ZYZY +
 0.035+0.000j IXXI +
 0.035-0.000j IYYI +
-0.035+0.000j ZXXZ +
-0.035+0.000j ZYYZ +
-0.021+0.000j IXXX +
 0.021-0.000j IXYY +
-0.021+0.000j IYXY +
-0.021+0.000j IYYX +
-0.019+0.000j XIII +
-0.019+0.000j XIZZ +
 0.019-0.000

We should also check that the ground state energy of the tapered Hamiltonian mathces that of the full system.

In [18]:
true_gs_energy, true_gs_vec = exact_gs_energy(molecule.H_q.to_sparse_matrix)
tapr_gs_energy, tapr_gs_vec = exact_gs_energy(ham_tap.to_sparse_matrix)

print(f'The ground state energy of the full system is {true_gs_energy},')
print(f'whereas for the tapered system we find the energy is {tapr_gs_energy}.')
print(f'The absolute error is {tapr_gs_energy-true_gs_energy}.')

The ground state energy of the full system is -1.87430197418373,
whereas for the tapered system we find the energy is -1.8643921454229446.
The absolute error is 0.009909828760785278.


Do they match? Depending on the molecule chosen, they might not! One can sometimes find that the Hartree-Fock state does not yield the correct symmetry sector, particularly in the strongly correlated regime.

In [19]:
hf_vec = np.eye(1,len(true_gs_vec),int(hf_string,2))
hf_overlap = np.square(np.abs(hf_vec.dot(true_gs_vec)))[0][0]

if hf_overlap < 1e-18:
    print('The Hartree-Fock state has no overlap with the true ground state!')
else:
    print('The Hartree-Fock state exhibits non-zero overlap with the true ground state.')
print(f'Overlap of the Hartree-Fock state with the true ground state: <HF|True GS> = {hf_overlap:.10f}')

The Hartree-Fock state has no overlap with the true ground state!
Overlap of the Hartree-Fock state with the true ground state: <HF|True GS> = 0.0000000000


If we instead take the dominant basis state in the true ground state, we should see that the energies do match in the resulting sector...

In [20]:
gs_psi = QuantumState.from_array(true_gs_vec).cleanup(zero_threshold=1e-5).sort()
print('The true ground state is:\n')
print(gs_psi); print()
print(f'Taking the dominant amplitude {gs_psi[0]} for sector selection...')

ham_tap_2 = taper_hamiltonian.taper_it(ref_state=gs_psi.state_matrix[0])
tap_gs_energy_2, tap_gs_2 = exact_gs_energy(ham_tap_2.to_sparse_matrix)

print(f'we obtain the sector {taper_hamiltonian.stabilizers.coeff_vec}, in which the ground state energy is {tap_gs_energy_2}.')
print(f'The absolute error is {tap_gs_energy_2-true_gs_energy}.')

The true ground state is:

 0.135+0.439j |11100100> +
-0.135-0.439j |11011000> +
-0.103-0.335j |01100110> +
-0.103-0.335j |10011001> +
-0.084-0.274j |00100111> +
 0.084+0.274j |00011011> +
 0.082+0.268j |10010110> +
 0.082+0.268j |01101001> +
 0.021+0.067j |01011010> +
 0.021+0.067j |10100101>

Taking the dominant amplitude  0.135+0.439j |11100100> for sector selection...
we obtain the sector [-1 -1  1  1], in which the ground state energy is -1.8743019741837192.
The absolute error is 1.0658141036401503e-14.


*The problem is...* 

we will not in general know how the basis states are distributed in the ground state!

The scalability of tapering is highly predicated on finding new approaches to identifying the correct symmetry sector.